In [46]:
import pandas as pd
import os

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("..\Data/csvs/basal_ganglia/regions")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

# create a map {} of name and ID and strength 


## Load csv
csv_file_path = "file:///%s" % get_csv_path("BAMS_region_mapping.csv")
df = pd.read_csv(csv_file_path)

class Region:
    
    def __init__(self, reg_id, name, abbr, desc):
        self.id = reg_id
        self.name = name
        self.abbreviation = abbr
        self.description = desc
        self.relates_to = []
        
    def __str__(self):
        pstr = str(self.id) + ", " + self.name + ", " + self.abbreviation
        for r in self.relates_to:
            pstr += " " + str(r)
        
        return pstr
        
        
    def setRelatesTo(self, region_ids):
        self.relates_to = region_ids
        
    
    
bams_regions = {}
relation_types = {"part of": 1, "overlapping": 2, "identical": 3, "includes": 4}

for index, row in df.iterrows():
    if(pd.isnull(row["All sorted"])):
        continue
    name_split =  row["All sorted"].split("(")
    name = name_split[0].strip()
    abbreviation = name_split[1][:-1]
    reg_id = index+1
    reg = Region(reg_id, name, abbreviation, row["Description"])
    
    # extract BG region ids
    rel_str = row["Relates to"]
    rel_str_splitted = rel_str.split()
    
    bg_ids = []
    for j in range(len(rel_str_splitted)):
        digit = rel_str_splitted[j]
        if (digit.isdigit() and j < len(rel_str_splitted)-1 and rel_str_splitted[j+1].startswith("WHS")):
            bg_ids.append(digit)
    
    rel_types = row["Relation type"].split(", ")
    
    ## Add bg reltation to Bams region class
    reg_relations = []
    for i in range(len(bg_ids)):
        reg_relations.append((bg_ids[i], rel_types[i]))
    
    reg.setRelatesTo(reg_relations)
    
    bams_regions[row["All sorted"]] = reg

bams_df_data = []
for key, region in bams_regions.items():
    bams_df_data.append([region.id, region.name, region.description])
    
bams_df = pd.DataFrame(bams_df_data, columns = ["id", "name", "desciption"]) 
bams_df.to_csv("..\Data/csvs/basal_ganglia/regions/bams2_regions.csv", index=1)

bams_mapping_data = []
for key, bamsR in bams_regions.items():
    for region in bamsR.relates_to:
        bams_mapping_data.append([region[0], bamsR.id, region[1]])
        
bams_mapping_df = pd.DataFrame(bams_mapping_data, columns = ["bg_id", "bams_id", "relation_type"]) 
bams_mapping_df.to_csv("..\Data/csvs/basal_ganglia/regions/bams2_mapping_regions.csv", index=1)    


#print(bams_df)
# return region_connectivity.csv


dataframes = []

for subdir, dirs, files in os.walk("..\Data/csvs/bams2_connectivity"):
    for file in files:
        path = "..\Data/csvs/bams2_connectivity/" + str(file)
        dataframes.append(pd.read_csv(path))

bg_connectivity_data = []

for connectivity_df in dataframes:
    for index, row in connectivity_df.iterrows():
        row["From"]
        row["To"]
        # get the mapped region
        
        try:
            bams_region_from = bams_regions[row["From"]]
            bams_region_to = bams_regions[row["To"]]

            for r1 in bams_region_from.relates_to:
                for r2 in bams_region_to.relates_to:
                    bg_connectivity_data.append([r1[0], r2[0], row["Strength"], row["Technique"], row["Description"], row["Reference"]])
        except:
            continue
        
bams_mapping_df = pd.DataFrame(bg_connectivity_data, columns = ["bg_id_from", "bg_id_to", "strength", "technique", "description", "reference"]) 
bams_mapping_df.to_csv("..\Data/csvs/basal_ganglia/regions/region_connectivity.csv", index=1)    
